#### Train a Convolutional Neural Network model for predicting cardiac arrhythmias.

#### Import necessary libraries.

In [1]:

import time
import pandas as pd
import tensorflow as tf
import numpy as np
import scipy.signal as signal
from collections import Counter
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import BatchNormalization, Dense, Flatten, Input, Conv1D, MaxPooling1D
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import PowerTransformer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.optimizers import Adam


#### Define a function to load the dataset.

In [2]:

def load_dataset(lead_name):
    # Loads the dataset using only the chosen lead.
    # Parameters:
    #    lead_name: Lead to be used.
    # Return:
    #    Dataframe loaded (Pandas dataframe)
    df = None
    column_names = ["idx", "ecg_id", lead_name, "arrhythmia_code"]
    dtypes = {"ecg_id": "str", lead_name : "float16", "arrhythmia_code" : "int16"}
    try:
        print("\nStart loading CSV file...")
        df = pd.read_csv("../dataset/csv_files/ecg_sph_dataset.csv", sep="|", dtype = dtypes, usecols = column_names)
        print("Finish loading CSV file.")
    except Exception as e:
        print("\nFail to load CSV file.")
        print("Error: {}".format(e))
    return df


#### Build a helper function to convert the records to the required format to perform a time series processing.

In [3]:

number_of_steps = 1250
number_of_features = 1
number_of_classes = 32

def get_new_columns_order(column_names_array):
    column_idx_count = 0
    new_array = np.zeros(len(column_names_array), dtype = int)
    for column_idx in range(0, (number_of_steps * 4)):
        for column_idx_2 in range(0, number_of_features):
            new_array[column_idx + column_idx_2 * (number_of_steps * 4)] = column_idx_count
            column_idx_count += 1
    return new_array

def build_time_window_structure(df, lead_name):
    # Splits the dataset into "time windows" to be used as a time series.
    # The function groups each 125 dataset records (CSV lines) into one record.
    # Parameters:
    #    df: Dataframe to be splitted.
    #    lead_name: Lead to be used.
    # Return:
    #    All time windows (np.array)
    #    All target values (np.array)
    print("\nStarting build_time_window_structure function...")
    df["idx"] = df["idx"] % (number_of_steps * 4)
    df_aux = df.pivot_table(index = "ecg_id", columns = "idx", values = [lead_name], aggfunc = "sum")
    new_columns = get_new_columns_order(df_aux.columns.values)
    df_aux.columns = list(new_columns)
    sorted_columns = sorted(df_aux.columns)
    df_modified = df_aux[sorted_columns]
    X_array = df_modified.values
    y_array = df["arrhythmia_code"].values
    y_array = y_array[::(number_of_steps * 4)]
    # Resample sample frequency to 125 hz.
    fs_original = 500 # Original frequency (Hz)
    fs_new = 125 # New frequency (Hz)
    downsampling_factor = int(fs_original / fs_new)
    nyquist_rate = fs_original / 2.0  # Nyquist rate
    cutoff_freq = fs_new / 2.0  # Cut off rate
    b, a = signal.butter(4, cutoff_freq / nyquist_rate, btype = "low")
    X_array_filtered = signal.filtfilt(b, a, X_array, axis = 1)
    X_array_125hz = X_array_filtered[:, ::downsampling_factor]
    print("\nShape of features: ", X_array_125hz.shape)
    print("Quantity os samples (labels): ", len(y_array))
    print("\nFinishing build_time_window_structure function.")
    return X_array_125hz, y_array


#### Define a function to remove classes with less than 6 samples.

In [4]:

def remove_classes_with_less_samples(X_array, y_array):
    # Remove classes with less than 6 samples.
    # Parameters:
    #    X_array: array of features.
    #    y_array: array of targets.
    # Return:
    #    Array of features (np.array)
    #    Array of targets (np.array)

    # Remove samples belonging to diagnostics 31, 37, 84, 87, 102, 143, 148, and 152 because these classes have less than 6 samples (SMOTE restriction).
    print("\nRemove classes with less than 6 samples.")
    removed_idx = np.where(np.isin(y_array, [31, 37, 84, 87, 102, 143, 148, 152]))[0]
    X_array = np.delete(X_array, removed_idx, axis = 0)
    y_array = np.delete(y_array, removed_idx, axis = 0)
    number_of_classes = 32
    # Generate a class number for each diagnostic code and replace y_array values.
    sorted_codes = sorted(set(y_array))
    dict_aux = {}
    for classes_idx in range(0, number_of_classes):
        dict_aux[classes_idx] = sorted_codes[classes_idx]
        y_array = [classes_idx if elem == sorted_codes[classes_idx] else elem for elem in y_array]
    y_array = np.array(y_array)
    print("\nShow classes identification:")
    for key, value in dict_aux.items():
        print(f"Class: {key} - Arrhythmia code: {value}")
    # Check for dataset balance.
    diagnostic_classes, count = np.unique(y_array, return_counts = True)
    percentage_by_class = [(i * 100 / np.sum(count)) for i in count]
    category_count = list(zip(diagnostic_classes, count, percentage_by_class))
    category_count.sort(key = lambda x: x[1], reverse = True)
    print("\nCheck for dataset balance:")
    for diagnostic_classes, count, percentage_by_class in category_count:
        print(f"Class = {diagnostic_classes:3.0f}   Qty = {count:8.0f}   Percentage = {percentage_by_class:2.2f} %")
    return X_array, y_array


#### Define a function for training a CNN model.

In [5]:

def apply_upsampling(X_array, y_array):
    # Apply SMOTE to balance the dataset.
    # Parameters:
    #    X_array (np.array): array of features values.
    #    y_array (np.array): array of target values.
    # Return:
    #    X_array (np.array): array of features values.
    #    y_array (np.array): array of target values.
    print("\nGenerating upsampling using SMOTE...")
    min_samples = min([sum(y_array == c) for c in set(y_array)])
    k_neighbors = min(5, min_samples - 1)
    smote = SMOTE(sampling_strategy = "auto", k_neighbors = k_neighbors, random_state = 42)
    X_array_res, y_array_res = smote.fit_resample(X_array, y_array)
    # Check for balance.
    diagnostic_codes, count = np.unique(y_array, return_counts = True)
    percentage_by_codes = [(i * 100 / np.sum(count)) for i in count]
    category_count = list(zip(diagnostic_codes, count, percentage_by_codes))
    category_count.sort(key = lambda x: x[1], reverse = True)
    # Use 6250 samples of each class for training.
    dict_samples_per_class = {}
    nr_samples_per_class = 6250
    for category_ids, count, percentage_of_categories in category_count:
        dict_samples_per_class[category_ids] = nr_samples_per_class
    rus = RandomUnderSampler(sampling_strategy = dict_samples_per_class, random_state = 42)
    X_train, y_train = rus.fit_resample(X_array_res, y_array_res)
    print("{} samples after upsampling.".format(len(y_train)))
    print(f"Class distribution for training after upsampling: {Counter(y_train)}")
    print("Finishing upsampling.\n")
    return X_train, y_train

def train_cnn_model(cnn_model, X, y, num_epochs, batch_size, validation_split, model_cfg_file):
    # Train a CNN model.
    # Parameters:
    #    cnn_model (Sequential): model to be trained.
    #    X (np.array): array of features values.
    #    y (np.array): array of target values.
    #    nun_folds (int): number of folds.
    #    num_epochs (int): number of epochs of training.
    #    batch_size (int): batch size.
    #    validation_split (float): percentage of instances for validation set.
    #    model_cfg_file (str): file to save the configuration model.
    # Returns:
    #    history (History object): history of training metrics.

    # Defining the number of folds (k-Fold).
    skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
    start_time = time.time()
    X_train = X
    y_train = y
    es = EarlyStopping(monitor = "val_loss", mode = "min", verbose = 1, patience = 10, restore_best_weights = True)
    train_accuracy_by_fold = []
    test_accuracy_by_fold = []
    fold_number = 1
    history_by_fold = []
    y_predclass_for_report = []
    y_testclass_for_report = []
    print("\nStarting training...")
    rb_scaler = PowerTransformer()
    for train_index, test_index in skf.split(X_train, y_train):
        print("\nTraining fold {}".format(fold_number))
        X_train_fold, y_train_fold = apply_upsampling(X_train[train_index], y_train[train_index])
        X_train_fold = rb_scaler.fit_transform(X_train_fold)
        X_train_fold = X_train_fold.reshape((X_train_fold.shape[0], number_of_steps, number_of_features))
        history = cnn_model.fit(X_train_fold, y_train_fold, validation_split = validation_split,
                                epochs = num_epochs, batch_size = batch_size, 
                                verbose = 1, callbacks = [es])
        _, train_accuracy = cnn_model.evaluate(X_train_fold, y_train_fold, verbose = 0)
        X_test_fold = rb_scaler.transform(X_train[test_index])
        X_test_reshaped = X_test_fold.reshape((X_test_fold.shape[0], number_of_steps, number_of_features))
        _, test_accuracy = cnn_model.evaluate(X_test_reshaped, y_train[test_index], verbose = 0)
        train_accuracy_by_fold.append(train_accuracy)
        test_accuracy_by_fold.append(test_accuracy)
        y_predclass_for_report.extend(np.argmax(cnn_model.predict(X_test_reshaped), axis = 1))
        y_testclass_for_report.extend(y_train[test_index])
        history_by_fold.append(history)
        fold_number += 1
    cnn_model.save("../modelconfig/" + model_cfg_file)
    elapsed_seconds = time.time() - start_time
    print("\nTime taken for training: ", time.strftime("%H:%M:%S", time.gmtime(elapsed_seconds)))
    print("\n")
    # Show metrics.
    for i in range(len(train_accuracy_by_fold)):
        print("Fold {} - Train Accuracy {:.4f} - Test Accuracy {:.4f}".format((i + 1), train_accuracy_by_fold[i],
                                                                              test_accuracy_by_fold[i]))
    print("\nMean Train Accuracy: {:.4f} - Std: {:.4f} ".format(np.mean(train_accuracy_by_fold),
                                                                np.std(train_accuracy_by_fold)))
    print("Mean Test Accuracy: {:.4f} - Std: {:.4f} ".format(np.mean(test_accuracy_by_fold),
                                                             np.std(test_accuracy_by_fold)))
    print("\nEvaluate other metrics:")
    print(classification_report(y_testclass_for_report, y_predclass_for_report, zero_division = 0))
    return history_by_fold


#### Define a function to build a version 1 of CNN model.

In [6]:

def create_v1():
    act_fuction = "relu"
    k_init = "he_uniform"
    model = Sequential()
    model.add(Input((number_of_steps, number_of_features)))
    model.add(Conv1D(filters = 8, kernel_size = 3, activation = act_fuction,
                     kernel_initializer = k_init))
    model.add(BatchNormalization())
    model.add(Conv1D(filters = 8, kernel_size = 3, activation = act_fuction, 
                     kernel_initializer = k_init))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Conv1D(filters = 16, kernel_size = 5, activation = act_fuction, 
                     kernel_initializer = k_init))
    model.add(BatchNormalization())
    model.add(Conv1D(filters = 16, kernel_size = 5, activation = act_fuction, 
                     kernel_initializer = k_init))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Flatten())
    model.add(Dense(128, activation = act_fuction, kernel_initializer = k_init))
    model.add(BatchNormalization())
    model.add(Dense(number_of_classes, activation = 'softmax'))
    opt = Adam(learning_rate = 0.001)
    model.summary()
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])
    return model


#### Lead I

In [7]:

# Build a "time window" structure to handle the dataset as a time series.
new_df = load_dataset("lead1")
X_array, y_array = build_time_window_structure(new_df, "lead1")
X_array, y_array = remove_classes_with_less_samples(X_array, y_array)

# Train the CNN model.
v1_model = create_v1()
v1_num_epochs = 300
v1_batch_size = 32
v1_validation_split = 0.01

training_history_v1 = train_cnn_model(v1_model, X_array, y_array, v1_num_epochs, v1_batch_size, v1_validation_split, "v1_model_lead1.keras")



Start loading CSV file...
Finish loading CSV file.

Starting build_time_window_structure function...

Shape of features:  (25770, 1250)
Quantity os samples (labels):  25770

Finishing build_time_window_structure function.

Remove classes with less than 6 samples.

Show classes identification:
Class: 0 - Arrhythmia code: 1
Class: 1 - Arrhythmia code: 21
Class: 2 - Arrhythmia code: 22
Class: 3 - Arrhythmia code: 23
Class: 4 - Arrhythmia code: 30
Class: 5 - Arrhythmia code: 36
Class: 6 - Arrhythmia code: 50
Class: 7 - Arrhythmia code: 51
Class: 8 - Arrhythmia code: 54
Class: 9 - Arrhythmia code: 60
Class: 10 - Arrhythmia code: 80
Class: 11 - Arrhythmia code: 82
Class: 12 - Arrhythmia code: 83
Class: 13 - Arrhythmia code: 88
Class: 14 - Arrhythmia code: 101
Class: 15 - Arrhythmia code: 104
Class: 16 - Arrhythmia code: 105
Class: 17 - Arrhythmia code: 106
Class: 18 - Arrhythmia code: 108
Class: 19 - Arrhythmia code: 120
Class: 20 - Arrhythmia code: 121
Class: 21 - Arrhythmia code: 125
Clas

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1248, 8)        │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1248, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1246, 8)        │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1246, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 623, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 619, 16)        │           656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 619, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 615, 16)        │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 615, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 307, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       628,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 635,880 (2.43 MB)

 Trainable params: 635,528 (2.42 MB)

 Non-trainable params: 352 (1.38 KB)


Starting training...

Training fold 1

Generating upsampling using SMOTE...


c:\DeveloperTools\python\3.11.0\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


200000 samples after upsampling.
Class distribution for training after upsampling: Counter({np.int64(0): 6250, np.int64(1): 6250, np.int64(2): 6250, np.int64(3): 6250, np.int64(4): 6250, np.int64(5): 6250, np.int64(6): 6250, np.int64(7): 6250, np.int64(8): 6250, np.int64(9): 6250, np.int64(10): 6250, np.int64(11): 6250, np.int64(12): 6250, np.int64(13): 6250, np.int64(14): 6250, np.int64(15): 6250, np.int64(16): 6250, np.int64(17): 6250, np.int64(18): 6250, np.int64(19): 6250, np.int64(20): 6250, np.int64(21): 6250, np.int64(22): 6250, np.int64(23): 6250, np.int64(24): 6250, np.int64(25): 6250, np.int64(26): 6250, np.int64(27): 6250, np.int64(28): 6250, np.int64(29): 6250, np.int64(30): 6250, np.int64(31): 6250})
Finishing upsampling.

Epoch 1/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 113s 17ms/step - accuracy: 0.8512 - loss: 0.5808 - val_accuracy: 1.0000 - val_loss: 1.0015e-04
Epoch 2/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 108s 17ms/step - accuracy: 0.9741 - loss: 0.0834 - val_accuracy: 1.0000 -

#### Lead II

In [7]:

# Build a "time window" structure to handle the dataset as a time series.
new_df = load_dataset("lead2")
X_array, y_array = build_time_window_structure(new_df, "lead2")
X_array, y_array = remove_classes_with_less_samples(X_array, y_array)

# Train the CNN model.
v1_model = create_v1()
v1_num_epochs = 300
v1_batch_size = 32
v1_validation_split = 0.01

training_history_v1 = train_cnn_model(v1_model, X_array, y_array, v1_num_epochs, v1_batch_size, v1_validation_split, "v1_model_lead2.keras")



Start loading CSV file...
Finish loading CSV file.

Starting build_time_window_structure function...

Shape of features:  (25770, 1250)
Quantity os samples (labels):  25770

Finishing build_time_window_structure function.

Remove classes with less than 6 samples.

Show classes identification:
Class: 0 - Arrhythmia code: 1
Class: 1 - Arrhythmia code: 21
Class: 2 - Arrhythmia code: 22
Class: 3 - Arrhythmia code: 23
Class: 4 - Arrhythmia code: 30
Class: 5 - Arrhythmia code: 36
Class: 6 - Arrhythmia code: 50
Class: 7 - Arrhythmia code: 51
Class: 8 - Arrhythmia code: 54
Class: 9 - Arrhythmia code: 60
Class: 10 - Arrhythmia code: 80
Class: 11 - Arrhythmia code: 82
Class: 12 - Arrhythmia code: 83
Class: 13 - Arrhythmia code: 88
Class: 14 - Arrhythmia code: 101
Class: 15 - Arrhythmia code: 104
Class: 16 - Arrhythmia code: 105
Class: 17 - Arrhythmia code: 106
Class: 18 - Arrhythmia code: 108
Class: 19 - Arrhythmia code: 120
Class: 20 - Arrhythmia code: 121
Class: 21 - Arrhythmia code: 125
Clas

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1248, 8)        │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1248, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1246, 8)        │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1246, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 623, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 619, 16)        │           656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 619, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 615, 16)        │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 615, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 307, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       628,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 635,880 (2.43 MB)

 Trainable params: 635,528 (2.42 MB)

 Non-trainable params: 352 (1.38 KB)


Starting training...

Training fold 1

Generating upsampling using SMOTE...


c:\DeveloperTools\python\3.11.0\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


200000 samples after upsampling.
Class distribution for training after upsampling: Counter({np.int64(0): 6250, np.int64(1): 6250, np.int64(2): 6250, np.int64(3): 6250, np.int64(4): 6250, np.int64(5): 6250, np.int64(6): 6250, np.int64(7): 6250, np.int64(8): 6250, np.int64(9): 6250, np.int64(10): 6250, np.int64(11): 6250, np.int64(12): 6250, np.int64(13): 6250, np.int64(14): 6250, np.int64(15): 6250, np.int64(16): 6250, np.int64(17): 6250, np.int64(18): 6250, np.int64(19): 6250, np.int64(20): 6250, np.int64(21): 6250, np.int64(22): 6250, np.int64(23): 6250, np.int64(24): 6250, np.int64(25): 6250, np.int64(26): 6250, np.int64(27): 6250, np.int64(28): 6250, np.int64(29): 6250, np.int64(30): 6250, np.int64(31): 6250})
Finishing upsampling.

Epoch 1/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 113s 18ms/step - accuracy: 0.8666 - loss: 0.5191 - val_accuracy: 1.0000 - val_loss: 7.0585e-05
Epoch 2/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 108s 17ms/step - accuracy: 0.9785 - loss: 0.0685 - val_accuracy: 1.0000 -

#### Lead III

In [7]:

# Build a "time window" structure to handle the dataset as a time series.
new_df = load_dataset("lead3")
X_array, y_array = build_time_window_structure(new_df, "lead3")
X_array, y_array = remove_classes_with_less_samples(X_array, y_array)

# Train the CNN model.
v1_model = create_v1()
v1_num_epochs = 300
v1_batch_size = 32
v1_validation_split = 0.01

training_history_v1 = train_cnn_model(v1_model, X_array, y_array, v1_num_epochs, v1_batch_size, v1_validation_split, "v1_model_lead3.keras")



Start loading CSV file...
Finish loading CSV file.

Starting build_time_window_structure function...

Shape of features:  (25770, 1250)
Quantity os samples (labels):  25770

Finishing build_time_window_structure function.

Remove classes with less than 6 samples.

Show classes identification:
Class: 0 - Arrhythmia code: 1
Class: 1 - Arrhythmia code: 21
Class: 2 - Arrhythmia code: 22
Class: 3 - Arrhythmia code: 23
Class: 4 - Arrhythmia code: 30
Class: 5 - Arrhythmia code: 36
Class: 6 - Arrhythmia code: 50
Class: 7 - Arrhythmia code: 51
Class: 8 - Arrhythmia code: 54
Class: 9 - Arrhythmia code: 60
Class: 10 - Arrhythmia code: 80
Class: 11 - Arrhythmia code: 82
Class: 12 - Arrhythmia code: 83
Class: 13 - Arrhythmia code: 88
Class: 14 - Arrhythmia code: 101
Class: 15 - Arrhythmia code: 104
Class: 16 - Arrhythmia code: 105
Class: 17 - Arrhythmia code: 106
Class: 18 - Arrhythmia code: 108
Class: 19 - Arrhythmia code: 120
Class: 20 - Arrhythmia code: 121
Class: 21 - Arrhythmia code: 125
Clas

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1248, 8)        │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1248, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1246, 8)        │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1246, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 623, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 619, 16)        │           656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 619, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 615, 16)        │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 615, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 307, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       628,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 635,880 (2.43 MB)

 Trainable params: 635,528 (2.42 MB)

 Non-trainable params: 352 (1.38 KB)


Starting training...

Training fold 1

Generating upsampling using SMOTE...


c:\DeveloperTools\python\3.11.0\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


200000 samples after upsampling.
Class distribution for training after upsampling: Counter({np.int64(0): 6250, np.int64(1): 6250, np.int64(2): 6250, np.int64(3): 6250, np.int64(4): 6250, np.int64(5): 6250, np.int64(6): 6250, np.int64(7): 6250, np.int64(8): 6250, np.int64(9): 6250, np.int64(10): 6250, np.int64(11): 6250, np.int64(12): 6250, np.int64(13): 6250, np.int64(14): 6250, np.int64(15): 6250, np.int64(16): 6250, np.int64(17): 6250, np.int64(18): 6250, np.int64(19): 6250, np.int64(20): 6250, np.int64(21): 6250, np.int64(22): 6250, np.int64(23): 6250, np.int64(24): 6250, np.int64(25): 6250, np.int64(26): 6250, np.int64(27): 6250, np.int64(28): 6250, np.int64(29): 6250, np.int64(30): 6250, np.int64(31): 6250})
Finishing upsampling.

Epoch 1/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 106s 17ms/step - accuracy: 0.8478 - loss: 0.5997 - val_accuracy: 1.0000 - val_loss: 2.2989e-04
Epoch 2/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 101s 16ms/step - accuracy: 0.9748 - loss: 0.0810 - val_accuracy: 1.0000 -

#### aVR

In [7]:

# Build a "time window" structure to handle the dataset as a time series.
new_df = load_dataset("aVR")
X_array, y_array = build_time_window_structure(new_df, "aVR")
X_array, y_array = remove_classes_with_less_samples(X_array, y_array)

# Train the CNN model.
v1_model = create_v1()
v1_num_epochs = 300
v1_batch_size = 32
v1_validation_split = 0.01

training_history_v1 = train_cnn_model(v1_model, X_array, y_array, v1_num_epochs, v1_batch_size, v1_validation_split, "v1_model_aVR.keras")



Start loading CSV file...
Finish loading CSV file.

Starting build_time_window_structure function...

Shape of features:  (25770, 1250)
Quantity os samples (labels):  25770

Finishing build_time_window_structure function.

Remove classes with less than 6 samples.

Show classes identification:
Class: 0 - Arrhythmia code: 1
Class: 1 - Arrhythmia code: 21
Class: 2 - Arrhythmia code: 22
Class: 3 - Arrhythmia code: 23
Class: 4 - Arrhythmia code: 30
Class: 5 - Arrhythmia code: 36
Class: 6 - Arrhythmia code: 50
Class: 7 - Arrhythmia code: 51
Class: 8 - Arrhythmia code: 54
Class: 9 - Arrhythmia code: 60
Class: 10 - Arrhythmia code: 80
Class: 11 - Arrhythmia code: 82
Class: 12 - Arrhythmia code: 83
Class: 13 - Arrhythmia code: 88
Class: 14 - Arrhythmia code: 101
Class: 15 - Arrhythmia code: 104
Class: 16 - Arrhythmia code: 105
Class: 17 - Arrhythmia code: 106
Class: 18 - Arrhythmia code: 108
Class: 19 - Arrhythmia code: 120
Class: 20 - Arrhythmia code: 121
Class: 21 - Arrhythmia code: 125
Clas

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1248, 8)        │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1248, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1246, 8)        │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1246, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 623, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 619, 16)        │           656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 619, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 615, 16)        │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 615, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 307, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       628,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 635,880 (2.43 MB)

 Trainable params: 635,528 (2.42 MB)

 Non-trainable params: 352 (1.38 KB)


Starting training...

Training fold 1

Generating upsampling using SMOTE...


c:\DeveloperTools\python\3.11.0\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


200000 samples after upsampling.
Class distribution for training after upsampling: Counter({np.int64(0): 6250, np.int64(1): 6250, np.int64(2): 6250, np.int64(3): 6250, np.int64(4): 6250, np.int64(5): 6250, np.int64(6): 6250, np.int64(7): 6250, np.int64(8): 6250, np.int64(9): 6250, np.int64(10): 6250, np.int64(11): 6250, np.int64(12): 6250, np.int64(13): 6250, np.int64(14): 6250, np.int64(15): 6250, np.int64(16): 6250, np.int64(17): 6250, np.int64(18): 6250, np.int64(19): 6250, np.int64(20): 6250, np.int64(21): 6250, np.int64(22): 6250, np.int64(23): 6250, np.int64(24): 6250, np.int64(25): 6250, np.int64(26): 6250, np.int64(27): 6250, np.int64(28): 6250, np.int64(29): 6250, np.int64(30): 6250, np.int64(31): 6250})
Finishing upsampling.

Epoch 1/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 105s 16ms/step - accuracy: 0.8709 - loss: 0.4969 - val_accuracy: 1.0000 - val_loss: 6.8303e-05
Epoch 2/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 100s 16ms/step - accuracy: 0.9780 - loss: 0.0697 - val_accuracy: 1.0000 -

#### aVL

In [10]:

# Build a "time window" structure to handle the dataset as a time series.
new_df = load_dataset("aVL")
X_array, y_array = build_time_window_structure(new_df, "aVL")
X_array, y_array = remove_classes_with_less_samples(X_array, y_array)

# Train the CNN model.
v1_model = create_v1()
v1_num_epochs = 300
v1_batch_size = 32
v1_validation_split = 0.01

training_history_v1 = train_cnn_model(v1_model, X_array, y_array, v1_num_epochs, v1_batch_size, v1_validation_split, "v1_model_aVL.keras")



Start loading CSV file...
Finish loading CSV file.

Starting build_time_window_structure function...

Shape of features:  (25770, 1250)
Quantity os samples (labels):  25770

Finishing build_time_window_structure function.

Remove classes with less than 6 samples.

Show classes identification:
Class: 0 - Arrhythmia code: 1
Class: 1 - Arrhythmia code: 21
Class: 2 - Arrhythmia code: 22
Class: 3 - Arrhythmia code: 23
Class: 4 - Arrhythmia code: 30
Class: 5 - Arrhythmia code: 36
Class: 6 - Arrhythmia code: 50
Class: 7 - Arrhythmia code: 51
Class: 8 - Arrhythmia code: 54
Class: 9 - Arrhythmia code: 60
Class: 10 - Arrhythmia code: 80
Class: 11 - Arrhythmia code: 82
Class: 12 - Arrhythmia code: 83
Class: 13 - Arrhythmia code: 88
Class: 14 - Arrhythmia code: 101
Class: 15 - Arrhythmia code: 104
Class: 16 - Arrhythmia code: 105
Class: 17 - Arrhythmia code: 106
Class: 18 - Arrhythmia code: 108
Class: 19 - Arrhythmia code: 120
Class: 20 - Arrhythmia code: 121
Class: 21 - Arrhythmia code: 125
Clas

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1248, 8)        │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1248, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1246, 8)        │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1246, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 623, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 619, 16)        │           656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 619, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 615, 16)        │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 615, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 307, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       628,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 635,880 (2.43 MB)

 Trainable params: 635,528 (2.42 MB)

 Non-trainable params: 352 (1.38 KB)


Starting training...

Training fold 1

Generating upsampling using SMOTE...


c:\DeveloperTools\python\3.11.0\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


200000 samples after upsampling.
Class distribution for training after upsampling: Counter({np.int64(0): 6250, np.int64(1): 6250, np.int64(2): 6250, np.int64(3): 6250, np.int64(4): 6250, np.int64(5): 6250, np.int64(6): 6250, np.int64(7): 6250, np.int64(8): 6250, np.int64(9): 6250, np.int64(10): 6250, np.int64(11): 6250, np.int64(12): 6250, np.int64(13): 6250, np.int64(14): 6250, np.int64(15): 6250, np.int64(16): 6250, np.int64(17): 6250, np.int64(18): 6250, np.int64(19): 6250, np.int64(20): 6250, np.int64(21): 6250, np.int64(22): 6250, np.int64(23): 6250, np.int64(24): 6250, np.int64(25): 6250, np.int64(26): 6250, np.int64(27): 6250, np.int64(28): 6250, np.int64(29): 6250, np.int64(30): 6250, np.int64(31): 6250})
Finishing upsampling.

Epoch 1/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 109s 17ms/step - accuracy: 0.8459 - loss: 0.6004 - val_accuracy: 1.0000 - val_loss: 4.2854e-05
Epoch 2/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 104s 17ms/step - accuracy: 0.9720 - loss: 0.0878 - val_accuracy: 1.0000 -

#### aVF

In [8]:

# Build a "time window" structure to handle the dataset as a time series.
new_df = load_dataset("aVF")
X_array, y_array = build_time_window_structure(new_df, "aVF")
X_array, y_array = remove_classes_with_less_samples(X_array, y_array)

# Train the CNN model.
v1_model = create_v1()
v1_num_epochs = 300
v1_batch_size = 32
v1_validation_split = 0.01

training_history_v1 = train_cnn_model(v1_model, X_array, y_array, v1_num_epochs, v1_batch_size, v1_validation_split, "v1_model_aVF.keras")



Start loading CSV file...
Finish loading CSV file.

Starting build_time_window_structure function...

Shape of features:  (25770, 1250)
Quantity os samples (labels):  25770

Finishing build_time_window_structure function.

Remove classes with less than 6 samples.

Show classes identification:
Class: 0 - Arrhythmia code: 1
Class: 1 - Arrhythmia code: 21
Class: 2 - Arrhythmia code: 22
Class: 3 - Arrhythmia code: 23
Class: 4 - Arrhythmia code: 30
Class: 5 - Arrhythmia code: 36
Class: 6 - Arrhythmia code: 50
Class: 7 - Arrhythmia code: 51
Class: 8 - Arrhythmia code: 54
Class: 9 - Arrhythmia code: 60
Class: 10 - Arrhythmia code: 80
Class: 11 - Arrhythmia code: 82
Class: 12 - Arrhythmia code: 83
Class: 13 - Arrhythmia code: 88
Class: 14 - Arrhythmia code: 101
Class: 15 - Arrhythmia code: 104
Class: 16 - Arrhythmia code: 105
Class: 17 - Arrhythmia code: 106
Class: 18 - Arrhythmia code: 108
Class: 19 - Arrhythmia code: 120
Class: 20 - Arrhythmia code: 121
Class: 21 - Arrhythmia code: 125
Clas

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 1248, 8)        │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 1248, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 1246, 8)        │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 1246, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 623, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 619, 16)        │           656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 619, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 615, 16)        │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 615, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 307, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       628,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 635,880 (2.43 MB)

 Trainable params: 635,528 (2.42 MB)

 Non-trainable params: 352 (1.38 KB)


Starting training...

Training fold 1

Generating upsampling using SMOTE...


c:\DeveloperTools\python\3.11.0\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


200000 samples after upsampling.
Class distribution for training after upsampling: Counter({np.int64(0): 6250, np.int64(1): 6250, np.int64(2): 6250, np.int64(3): 6250, np.int64(4): 6250, np.int64(5): 6250, np.int64(6): 6250, np.int64(7): 6250, np.int64(8): 6250, np.int64(9): 6250, np.int64(10): 6250, np.int64(11): 6250, np.int64(12): 6250, np.int64(13): 6250, np.int64(14): 6250, np.int64(15): 6250, np.int64(16): 6250, np.int64(17): 6250, np.int64(18): 6250, np.int64(19): 6250, np.int64(20): 6250, np.int64(21): 6250, np.int64(22): 6250, np.int64(23): 6250, np.int64(24): 6250, np.int64(25): 6250, np.int64(26): 6250, np.int64(27): 6250, np.int64(28): 6250, np.int64(29): 6250, np.int64(30): 6250, np.int64(31): 6250})
Finishing upsampling.

Epoch 1/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 178s 28ms/step - accuracy: 0.8506 - loss: 0.5835 - val_accuracy: 1.0000 - val_loss: 8.0297e-04
Epoch 2/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 175s 28ms/step - accuracy: 0.9761 - loss: 0.0769 - val_accuracy: 1.0000 -

#### V1

In [7]:

# Build a "time window" structure to handle the dataset as a time series.
new_df = load_dataset("V1")
X_array, y_array = build_time_window_structure(new_df, "V1")
X_array, y_array = remove_classes_with_less_samples(X_array, y_array)

# Train the CNN model.
v1_model = create_v1()
v1_num_epochs = 300
v1_batch_size = 32
v1_validation_split = 0.01

training_history_v1 = train_cnn_model(v1_model, X_array, y_array, v1_num_epochs, v1_batch_size, v1_validation_split, "v1_model_V1.keras")



Start loading CSV file...
Finish loading CSV file.

Starting build_time_window_structure function...

Shape of features:  (25770, 1250)
Quantity os samples (labels):  25770

Finishing build_time_window_structure function.

Remove classes with less than 6 samples.

Show classes identification:
Class: 0 - Arrhythmia code: 1
Class: 1 - Arrhythmia code: 21
Class: 2 - Arrhythmia code: 22
Class: 3 - Arrhythmia code: 23
Class: 4 - Arrhythmia code: 30
Class: 5 - Arrhythmia code: 36
Class: 6 - Arrhythmia code: 50
Class: 7 - Arrhythmia code: 51
Class: 8 - Arrhythmia code: 54
Class: 9 - Arrhythmia code: 60
Class: 10 - Arrhythmia code: 80
Class: 11 - Arrhythmia code: 82
Class: 12 - Arrhythmia code: 83
Class: 13 - Arrhythmia code: 88
Class: 14 - Arrhythmia code: 101
Class: 15 - Arrhythmia code: 104
Class: 16 - Arrhythmia code: 105
Class: 17 - Arrhythmia code: 106
Class: 18 - Arrhythmia code: 108
Class: 19 - Arrhythmia code: 120
Class: 20 - Arrhythmia code: 121
Class: 21 - Arrhythmia code: 125
Clas

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1248, 8)        │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1248, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1246, 8)        │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1246, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 623, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 619, 16)        │           656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 619, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 615, 16)        │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 615, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 307, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       628,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 635,880 (2.43 MB)

 Trainable params: 635,528 (2.42 MB)

 Non-trainable params: 352 (1.38 KB)


Starting training...

Training fold 1

Generating upsampling using SMOTE...


c:\DeveloperTools\python\3.11.0\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


200000 samples after upsampling.
Class distribution for training after upsampling: Counter({np.int64(0): 6250, np.int64(1): 6250, np.int64(2): 6250, np.int64(3): 6250, np.int64(4): 6250, np.int64(5): 6250, np.int64(6): 6250, np.int64(7): 6250, np.int64(8): 6250, np.int64(9): 6250, np.int64(10): 6250, np.int64(11): 6250, np.int64(12): 6250, np.int64(13): 6250, np.int64(14): 6250, np.int64(15): 6250, np.int64(16): 6250, np.int64(17): 6250, np.int64(18): 6250, np.int64(19): 6250, np.int64(20): 6250, np.int64(21): 6250, np.int64(22): 6250, np.int64(23): 6250, np.int64(24): 6250, np.int64(25): 6250, np.int64(26): 6250, np.int64(27): 6250, np.int64(28): 6250, np.int64(29): 6250, np.int64(30): 6250, np.int64(31): 6250})
Finishing upsampling.

Epoch 1/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 107s 17ms/step - accuracy: 0.8505 - loss: 0.5826 - val_accuracy: 1.0000 - val_loss: 0.0010
Epoch 2/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 101s 16ms/step - accuracy: 0.9714 - loss: 0.0915 - val_accuracy: 1.0000 - val

#### V2

In [7]:

# Build a "time window" structure to handle the dataset as a time series.
new_df = load_dataset("V2")
X_array, y_array = build_time_window_structure(new_df, "V2")
X_array, y_array = remove_classes_with_less_samples(X_array, y_array)

# Train the CNN model.
v1_model = create_v1()
v1_num_epochs = 300
v1_batch_size = 32
v1_validation_split = 0.01

training_history_v1 = train_cnn_model(v1_model, X_array, y_array, v1_num_epochs, v1_batch_size, v1_validation_split, "v1_model_V2.keras")



Start loading CSV file...
Finish loading CSV file.

Starting build_time_window_structure function...

Shape of features:  (25770, 1250)
Quantity os samples (labels):  25770

Finishing build_time_window_structure function.

Remove classes with less than 6 samples.

Show classes identification:
Class: 0 - Arrhythmia code: 1
Class: 1 - Arrhythmia code: 21
Class: 2 - Arrhythmia code: 22
Class: 3 - Arrhythmia code: 23
Class: 4 - Arrhythmia code: 30
Class: 5 - Arrhythmia code: 36
Class: 6 - Arrhythmia code: 50
Class: 7 - Arrhythmia code: 51
Class: 8 - Arrhythmia code: 54
Class: 9 - Arrhythmia code: 60
Class: 10 - Arrhythmia code: 80
Class: 11 - Arrhythmia code: 82
Class: 12 - Arrhythmia code: 83
Class: 13 - Arrhythmia code: 88
Class: 14 - Arrhythmia code: 101
Class: 15 - Arrhythmia code: 104
Class: 16 - Arrhythmia code: 105
Class: 17 - Arrhythmia code: 106
Class: 18 - Arrhythmia code: 108
Class: 19 - Arrhythmia code: 120
Class: 20 - Arrhythmia code: 121
Class: 21 - Arrhythmia code: 125
Clas

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1248, 8)        │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1248, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1246, 8)        │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1246, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 623, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 619, 16)        │           656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 619, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 615, 16)        │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 615, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 307, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       628,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 635,880 (2.43 MB)

 Trainable params: 635,528 (2.42 MB)

 Non-trainable params: 352 (1.38 KB)


Starting training...

Training fold 1

Generating upsampling using SMOTE...


c:\DeveloperTools\python\3.11.0\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


200000 samples after upsampling.
Class distribution for training after upsampling: Counter({np.int64(0): 6250, np.int64(1): 6250, np.int64(2): 6250, np.int64(3): 6250, np.int64(4): 6250, np.int64(5): 6250, np.int64(6): 6250, np.int64(7): 6250, np.int64(8): 6250, np.int64(9): 6250, np.int64(10): 6250, np.int64(11): 6250, np.int64(12): 6250, np.int64(13): 6250, np.int64(14): 6250, np.int64(15): 6250, np.int64(16): 6250, np.int64(17): 6250, np.int64(18): 6250, np.int64(19): 6250, np.int64(20): 6250, np.int64(21): 6250, np.int64(22): 6250, np.int64(23): 6250, np.int64(24): 6250, np.int64(25): 6250, np.int64(26): 6250, np.int64(27): 6250, np.int64(28): 6250, np.int64(29): 6250, np.int64(30): 6250, np.int64(31): 6250})
Finishing upsampling.

Epoch 1/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 108s 17ms/step - accuracy: 0.8528 - loss: 0.5718 - val_accuracy: 1.0000 - val_loss: 2.7154e-04
Epoch 2/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 102s 17ms/step - accuracy: 0.9724 - loss: 0.0879 - val_accuracy: 1.0000 -

#### V3

In [7]:

# Build a "time window" structure to handle the dataset as a time series.
new_df = load_dataset("V3")
X_array, y_array = build_time_window_structure(new_df, "V3")
X_array, y_array = remove_classes_with_less_samples(X_array, y_array)

# Train the CNN model.
v1_model = create_v1()
v1_num_epochs = 300
v1_batch_size = 32
v1_validation_split = 0.01

training_history_v1 = train_cnn_model(v1_model, X_array, y_array, v1_num_epochs, v1_batch_size, v1_validation_split, "v1_model_V3.keras")



Start loading CSV file...
Finish loading CSV file.

Starting build_time_window_structure function...

Shape of features:  (25770, 1250)
Quantity os samples (labels):  25770

Finishing build_time_window_structure function.

Remove classes with less than 6 samples.

Show classes identification:
Class: 0 - Arrhythmia code: 1
Class: 1 - Arrhythmia code: 21
Class: 2 - Arrhythmia code: 22
Class: 3 - Arrhythmia code: 23
Class: 4 - Arrhythmia code: 30
Class: 5 - Arrhythmia code: 36
Class: 6 - Arrhythmia code: 50
Class: 7 - Arrhythmia code: 51
Class: 8 - Arrhythmia code: 54
Class: 9 - Arrhythmia code: 60
Class: 10 - Arrhythmia code: 80
Class: 11 - Arrhythmia code: 82
Class: 12 - Arrhythmia code: 83
Class: 13 - Arrhythmia code: 88
Class: 14 - Arrhythmia code: 101
Class: 15 - Arrhythmia code: 104
Class: 16 - Arrhythmia code: 105
Class: 17 - Arrhythmia code: 106
Class: 18 - Arrhythmia code: 108
Class: 19 - Arrhythmia code: 120
Class: 20 - Arrhythmia code: 121
Class: 21 - Arrhythmia code: 125
Clas

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1248, 8)        │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1248, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1246, 8)        │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1246, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 623, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 619, 16)        │           656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 619, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 615, 16)        │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 615, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 307, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       628,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 635,880 (2.43 MB)

 Trainable params: 635,528 (2.42 MB)

 Non-trainable params: 352 (1.38 KB)


Starting training...

Training fold 1

Generating upsampling using SMOTE...


c:\DeveloperTools\python\3.11.0\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


200000 samples after upsampling.
Class distribution for training after upsampling: Counter({np.int64(0): 6250, np.int64(1): 6250, np.int64(2): 6250, np.int64(3): 6250, np.int64(4): 6250, np.int64(5): 6250, np.int64(6): 6250, np.int64(7): 6250, np.int64(8): 6250, np.int64(9): 6250, np.int64(10): 6250, np.int64(11): 6250, np.int64(12): 6250, np.int64(13): 6250, np.int64(14): 6250, np.int64(15): 6250, np.int64(16): 6250, np.int64(17): 6250, np.int64(18): 6250, np.int64(19): 6250, np.int64(20): 6250, np.int64(21): 6250, np.int64(22): 6250, np.int64(23): 6250, np.int64(24): 6250, np.int64(25): 6250, np.int64(26): 6250, np.int64(27): 6250, np.int64(28): 6250, np.int64(29): 6250, np.int64(30): 6250, np.int64(31): 6250})
Finishing upsampling.

Epoch 1/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 115s 18ms/step - accuracy: 0.8614 - loss: 0.5435 - val_accuracy: 1.0000 - val_loss: 4.5996e-05
Epoch 2/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 113s 18ms/step - accuracy: 0.9767 - loss: 0.0764 - val_accuracy: 1.0000 -

#### V4

In [7]:

# Build a "time window" structure to handle the dataset as a time series.
new_df = load_dataset("V4")
X_array, y_array = build_time_window_structure(new_df, "V4")
X_array, y_array = remove_classes_with_less_samples(X_array, y_array)

# Train the CNN model.
v1_model = create_v1()
v1_num_epochs = 300
v1_batch_size = 32
v1_validation_split = 0.01

training_history_v1 = train_cnn_model(v1_model, X_array, y_array, v1_num_epochs, v1_batch_size, v1_validation_split, "v1_model_V4.keras")



Start loading CSV file...
Finish loading CSV file.

Starting build_time_window_structure function...

Shape of features:  (25770, 1250)
Quantity os samples (labels):  25770

Finishing build_time_window_structure function.

Remove classes with less than 6 samples.

Show classes identification:
Class: 0 - Arrhythmia code: 1
Class: 1 - Arrhythmia code: 21
Class: 2 - Arrhythmia code: 22
Class: 3 - Arrhythmia code: 23
Class: 4 - Arrhythmia code: 30
Class: 5 - Arrhythmia code: 36
Class: 6 - Arrhythmia code: 50
Class: 7 - Arrhythmia code: 51
Class: 8 - Arrhythmia code: 54
Class: 9 - Arrhythmia code: 60
Class: 10 - Arrhythmia code: 80
Class: 11 - Arrhythmia code: 82
Class: 12 - Arrhythmia code: 83
Class: 13 - Arrhythmia code: 88
Class: 14 - Arrhythmia code: 101
Class: 15 - Arrhythmia code: 104
Class: 16 - Arrhythmia code: 105
Class: 17 - Arrhythmia code: 106
Class: 18 - Arrhythmia code: 108
Class: 19 - Arrhythmia code: 120
Class: 20 - Arrhythmia code: 121
Class: 21 - Arrhythmia code: 125
Clas

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1248, 8)        │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1248, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1246, 8)        │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1246, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 623, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 619, 16)        │           656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 619, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 615, 16)        │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 615, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 307, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       628,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 635,880 (2.43 MB)

 Trainable params: 635,528 (2.42 MB)

 Non-trainable params: 352 (1.38 KB)


Starting training...

Training fold 1

Generating upsampling using SMOTE...


c:\DeveloperTools\python\3.11.0\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


200000 samples after upsampling.
Class distribution for training after upsampling: Counter({np.int64(0): 6250, np.int64(1): 6250, np.int64(2): 6250, np.int64(3): 6250, np.int64(4): 6250, np.int64(5): 6250, np.int64(6): 6250, np.int64(7): 6250, np.int64(8): 6250, np.int64(9): 6250, np.int64(10): 6250, np.int64(11): 6250, np.int64(12): 6250, np.int64(13): 6250, np.int64(14): 6250, np.int64(15): 6250, np.int64(16): 6250, np.int64(17): 6250, np.int64(18): 6250, np.int64(19): 6250, np.int64(20): 6250, np.int64(21): 6250, np.int64(22): 6250, np.int64(23): 6250, np.int64(24): 6250, np.int64(25): 6250, np.int64(26): 6250, np.int64(27): 6250, np.int64(28): 6250, np.int64(29): 6250, np.int64(30): 6250, np.int64(31): 6250})
Finishing upsampling.

Epoch 1/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 111s 17ms/step - accuracy: 0.8706 - loss: 0.5080 - val_accuracy: 1.0000 - val_loss: 3.1178e-05
Epoch 2/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 104s 17ms/step - accuracy: 0.9761 - loss: 0.0761 - val_accuracy: 1.0000 -

#### V5

In [7]:

# Build a "time window" structure to handle the dataset as a time series.
new_df = load_dataset("V5")
X_array, y_array = build_time_window_structure(new_df, "V5")
X_array, y_array = remove_classes_with_less_samples(X_array, y_array)

# Train the CNN model.
v1_model = create_v1()
v1_num_epochs = 300
v1_batch_size = 32
v1_validation_split = 0.01

training_history_v1 = train_cnn_model(v1_model, X_array, y_array, v1_num_epochs, v1_batch_size, v1_validation_split, "v1_model_V5.keras")



Start loading CSV file...
Finish loading CSV file.

Starting build_time_window_structure function...

Shape of features:  (25770, 1250)
Quantity os samples (labels):  25770

Finishing build_time_window_structure function.

Remove classes with less than 6 samples.

Show classes identification:
Class: 0 - Arrhythmia code: 1
Class: 1 - Arrhythmia code: 21
Class: 2 - Arrhythmia code: 22
Class: 3 - Arrhythmia code: 23
Class: 4 - Arrhythmia code: 30
Class: 5 - Arrhythmia code: 36
Class: 6 - Arrhythmia code: 50
Class: 7 - Arrhythmia code: 51
Class: 8 - Arrhythmia code: 54
Class: 9 - Arrhythmia code: 60
Class: 10 - Arrhythmia code: 80
Class: 11 - Arrhythmia code: 82
Class: 12 - Arrhythmia code: 83
Class: 13 - Arrhythmia code: 88
Class: 14 - Arrhythmia code: 101
Class: 15 - Arrhythmia code: 104
Class: 16 - Arrhythmia code: 105
Class: 17 - Arrhythmia code: 106
Class: 18 - Arrhythmia code: 108
Class: 19 - Arrhythmia code: 120
Class: 20 - Arrhythmia code: 121
Class: 21 - Arrhythmia code: 125
Clas

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1248, 8)        │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1248, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1246, 8)        │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1246, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 623, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 619, 16)        │           656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 619, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 615, 16)        │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 615, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 307, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       628,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 635,880 (2.43 MB)

 Trainable params: 635,528 (2.42 MB)

 Non-trainable params: 352 (1.38 KB)


Starting training...

Training fold 1

Generating upsampling using SMOTE...


c:\DeveloperTools\python\3.11.0\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


200000 samples after upsampling.
Class distribution for training after upsampling: Counter({np.int64(0): 6250, np.int64(1): 6250, np.int64(2): 6250, np.int64(3): 6250, np.int64(4): 6250, np.int64(5): 6250, np.int64(6): 6250, np.int64(7): 6250, np.int64(8): 6250, np.int64(9): 6250, np.int64(10): 6250, np.int64(11): 6250, np.int64(12): 6250, np.int64(13): 6250, np.int64(14): 6250, np.int64(15): 6250, np.int64(16): 6250, np.int64(17): 6250, np.int64(18): 6250, np.int64(19): 6250, np.int64(20): 6250, np.int64(21): 6250, np.int64(22): 6250, np.int64(23): 6250, np.int64(24): 6250, np.int64(25): 6250, np.int64(26): 6250, np.int64(27): 6250, np.int64(28): 6250, np.int64(29): 6250, np.int64(30): 6250, np.int64(31): 6250})
Finishing upsampling.

Epoch 1/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 101s 16ms/step - accuracy: 0.8720 - loss: 0.5014 - val_accuracy: 1.0000 - val_loss: 6.2636e-05
Epoch 2/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 105s 17ms/step - accuracy: 0.9789 - loss: 0.0687 - val_accuracy: 1.0000 -

#### V6

In [7]:

# Build a "time window" structure to handle the dataset as a time series.
new_df = load_dataset("V6")
X_array, y_array = build_time_window_structure(new_df, "V6")
X_array, y_array = remove_classes_with_less_samples(X_array, y_array)

# Train the CNN model.
v1_model = create_v1()
v1_num_epochs = 300
v1_batch_size = 32
v1_validation_split = 0.01

training_history_v1 = train_cnn_model(v1_model, X_array, y_array, v1_num_epochs, v1_batch_size, v1_validation_split, "v1_model_V6.keras")



Start loading CSV file...
Finish loading CSV file.

Starting build_time_window_structure function...

Shape of features:  (25770, 1250)
Quantity os samples (labels):  25770

Finishing build_time_window_structure function.

Remove classes with less than 6 samples.

Show classes identification:
Class: 0 - Arrhythmia code: 1
Class: 1 - Arrhythmia code: 21
Class: 2 - Arrhythmia code: 22
Class: 3 - Arrhythmia code: 23
Class: 4 - Arrhythmia code: 30
Class: 5 - Arrhythmia code: 36
Class: 6 - Arrhythmia code: 50
Class: 7 - Arrhythmia code: 51
Class: 8 - Arrhythmia code: 54
Class: 9 - Arrhythmia code: 60
Class: 10 - Arrhythmia code: 80
Class: 11 - Arrhythmia code: 82
Class: 12 - Arrhythmia code: 83
Class: 13 - Arrhythmia code: 88
Class: 14 - Arrhythmia code: 101
Class: 15 - Arrhythmia code: 104
Class: 16 - Arrhythmia code: 105
Class: 17 - Arrhythmia code: 106
Class: 18 - Arrhythmia code: 108
Class: 19 - Arrhythmia code: 120
Class: 20 - Arrhythmia code: 121
Class: 21 - Arrhythmia code: 125
Clas

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1248, 8)        │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1248, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1246, 8)        │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1246, 8)        │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 623, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 619, 16)        │           656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 619, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 615, 16)        │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 615, 16)        │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 307, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       628,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 635,880 (2.43 MB)

 Trainable params: 635,528 (2.42 MB)

 Non-trainable params: 352 (1.38 KB)


Starting training...

Training fold 1

Generating upsampling using SMOTE...


c:\DeveloperTools\python\3.11.0\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


200000 samples after upsampling.
Class distribution for training after upsampling: Counter({np.int64(0): 6250, np.int64(1): 6250, np.int64(2): 6250, np.int64(3): 6250, np.int64(4): 6250, np.int64(5): 6250, np.int64(6): 6250, np.int64(7): 6250, np.int64(8): 6250, np.int64(9): 6250, np.int64(10): 6250, np.int64(11): 6250, np.int64(12): 6250, np.int64(13): 6250, np.int64(14): 6250, np.int64(15): 6250, np.int64(16): 6250, np.int64(17): 6250, np.int64(18): 6250, np.int64(19): 6250, np.int64(20): 6250, np.int64(21): 6250, np.int64(22): 6250, np.int64(23): 6250, np.int64(24): 6250, np.int64(25): 6250, np.int64(26): 6250, np.int64(27): 6250, np.int64(28): 6250, np.int64(29): 6250, np.int64(30): 6250, np.int64(31): 6250})
Finishing upsampling.

Epoch 1/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 107s 17ms/step - accuracy: 0.8635 - loss: 0.5306 - val_accuracy: 1.0000 - val_loss: 0.0056
Epoch 2/300
6188/6188 ━━━━━━━━━━━━━━━━━━━━ 103s 17ms/step - accuracy: 0.9746 - loss: 0.0825 - val_accuracy: 0.9805 - val